# Customer Segmentation & Feature Engineering
This notebook builds engineered features and segments customers (Bronze/Silver/Gold).

In [ ]:
import pandas as pd, numpy as np
df = pd.read_csv('../data/Customer_Dataset.csv')
df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].median())
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].median())
df['SPEND'] = df['PURCHASES'] + df['ONEOFF_PURCHASES'] + df['INSTALLMENTS_PURCHASES']
df['SPEND_TO_LIMIT'] = (df['SPEND'] / df['CREDIT_LIMIT']).replace([np.inf, -np.inf], 0).fillna(0)
df['PAYMENT_TO_SPEND'] = (df['PAYMENTS'] / df['SPEND']).replace([np.inf, -np.inf], np.nan)
df['ADVANCE_RATIO'] = (df['CASH_ADVANCE'] / df['SPEND']).replace([np.inf, -np.inf], 0).fillna(0)
df['FREQ_SCORE'] = df['PURCHASES_FREQUENCY'] + df['ONEOFF_PURCHASES_FREQ'] + df['PURCHASES_INSTALLMENTS_FREQUENCY']
df['LTV_SCORE_RAW'] = 0.5*df['SPEND_TO_LIMIT'] + 0.3*df['FREQ_SCORE'] + 0.2*df['PRC_FULL_PAYMENT'].fillna(0)
q = df['LTV_SCORE_RAW'].quantile([0.33, 0.66]).values
def seg(x):
    return 'Bronze' if x<=q[0] else ('Silver' if x<=q[1] else 'Gold')
df['SEGMENT'] = df['LTV_SCORE_RAW'].apply(seg)
df.to_csv('../data/Customer_Dataset_engineered.csv', index=False)
df.head()